## Implementation of Neural Network with Natural-optimization-algorithms instead of Gradient descent

In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

#### Importing Nature-inspired-optims algorithms

In [2]:
from model.AssignWeights import assign_weights
from model.GA import *
from model.PSO import *
from model.ACO import *

## Dataset preparation

In [3]:
df = pd.read_csv('data/Bank_Personal_Loan_Modelling.csv')

In [4]:
df.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [5]:
df.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

In [6]:
X = df[['Age', 'Experience', 'Income', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Securities Account',
       'CD Account', 'Online', 'CreditCard']]
y = df[['Personal Loan']]

In [7]:
X.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,1,0,0,0
1,45,19,34,3,1.5,1,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,1


In [8]:
y.head()

,Personal Loan
0,0
1,0
2,0
3,0
4,0


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, stratify=y.values, random_state=1)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = X_train.T
X_test = X_test.T

## Model and loss

In [11]:
class Net():
    def __init__(self, in_features=1, out_features=1):
        self.W1 = np.random.rand(8,11)
        self.B1 = np.random.rand(8,1)
        self.W2 = np.random.rand(4,8)
        self.B2 = np.random.rand(4,1)
        self.W3 = np.random.rand(1,4)
        self.B3 = np.random.rand(1,1)
    def forward(self, X):
        output = self.W1 @ X + self.B1
        output = self.W2 @ output + self.B2
        output = self.W3 @ output + self.B3
        return output
    def predict(self, X):
        return self.forward(X)
model = Net()

In [12]:
model

In [13]:
def train_aco(max_iter, graph, X, Y, model, ph, size=30):
    final_min_cost = None
    final_paths = None
    all_costs = None
    
    all_time_best_cost = 100
    all_time_best_weights = None
    for i in range(max_iter):
        print(f'Iteration: {i+1}')
        ph, costs, paths = ACO(graph, X, Y, model, ph, size) # 80 ants
        # When several paths(set a threshold count) has same cost --> Optimal path has been found, then terminate
        
        if min(costs) < all_time_best_cost:
            all_time_best_cost = min(costs)
            all_time_best_weights = paths[np.where(costs == min(costs))[0][0]]
        
        print('Costs')
        print(costs)
#         unique_cost, counts = np.unique(costs, return_counts=True)
#         count_sort_ind = np.argsort(-counts)
#         counts = counts[count_sort_ind]
#         unique_cost = unique_cost[count_sort_ind]
#         print(unique_cost, counts)
#         print('--------------------------------------------')
#         all_costs = costs
#         final_min_cost = min(unique_cost)
#         final_paths = paths
        print()
        print()
    return all_time_best_cost, all_time_best_weights
    

In [14]:
def train_pso(model, X, Y, max_iter, particle_pos, particle_vel, particle_best_pos, particle_best_cost, global_best_cost):
    losses_iteration = []
    final_particle = []
    least_loss = None
    for i in range(max_iter):
        print(f'Iteration: {i+1}')
        particle_pos, particle_vel, particle_best_pos, particle_best_cost, global_best_cost = PCO(model, particle_pos, particle_vel, particle_best_pos, particle_best_cost, global_best_cost, X, Y,  limit)
        print(f'Global best: {global_best_cost}')
        print(f'Personal best of all {n} particles:')
        print(particle_best_cost)
        print('------------------------------------------------')
        losses_iteration.append(global_best_cost)
        final_particle = particle_best_pos
        least_loss = particle_best_cost
        
    return losses_iteration, final_particle, least_loss

In [15]:
def train_gen(model, pop, generation, X, y, crossover_rate, mutation_rate, k):
    losses_generation = []
    losses = None
    for i in range(generation):
        
        pop, losses = GeneticAlgorithm(model, pop,X, y, crossover_rate, mutation_rate, k)
        
        # Displaying loss at each generation
        print(min(losses))
        final_loss = min(losses)
        losses_generation.append(min(losses)) 

    return losses_generation, pop, losses
    

In [19]:
Alg = 2

if Alg == 1:
    # Genetic Algorithm
    pop = initlize_population(200, 137) # Total 137 parameters
    losses, pop, final_loss = train_gen(
        model, pop, 100, X_train, y_train,
        crossover_rate=0.3, mutation_rate=0.6, k=100
    )
elif Alg == 2:
    # Particle Swarm Optimisation
    max_iter = 100
    
    # Particle size
    n = 20
    
    # Total weights in NN
    dim = 137
    
    # Dont allow it to exceed this limit
    limit = [-2,2]

    particle_pos, particle_vel, particle_best_pos, particle_best_cost, global_best_cost = initialize(n,dim)

    losses_iteration, final_particle, final_loss = train_pso(model, X_train, y_train, 100, 
              particle_pos, particle_vel, particle_best_pos, 
              particle_best_cost, global_best_cost)

else:
    # Ant Colony Optimization
    v = 10
    graph = cost_graph(v, s=0.2)
    
    # Enhance the diversity of cost graph
    r = np.random.randint(5, 8)
    print(r)
    for i in range(r):
        s = np.random.uniform(low=1.0, high=2.5)
        f = np.random.uniform(low=0.5, high=0.7)
        graph = random_fill(graph, s,f)
    
    ph = pheromone_graph(v)
    max_iter = 50
    atbc, atbw = train_aco(max_iter, graph, X_train, y_train, model, ph)
    
    

Iteration: 1
Global best: 0.6066604852676392
Personal best of all 20 particles:
[0.67380643 0.63332719 0.71983725 0.66246265 0.63082564 0.68983442
 0.72114855 0.712403   0.64681774 0.68033767 0.60666049 0.66523421
 0.7165488  0.73212826 0.67981929 0.61486959 0.67603892 0.61612934
 0.69517636 0.6399892 ]
------------------------------------------------
Iteration: 2
Global best: 0.6066604852676392
Personal best of all 20 particles:
[0.63316238 0.61422485 0.65131146 0.62922031 0.61654347 0.63399577
 0.64999133 0.64473504 0.61969113 0.63905305 0.60666049 0.63004327
 0.64575261 0.65269083 0.63919765 0.61159837 0.62746835 0.61244702
 0.64156836 0.61960834]
------------------------------------------------
Iteration: 3
Global best: 0.5688102841377258
Personal best of all 20 particles:
[0.59062153 0.6003288  0.56881028 0.58878791 0.59834814 0.58615202
 0.57277775 0.57980067 0.59809011 0.57891822 0.60666049 0.58996975
 0.58117974 0.57248986 0.57915968 0.60091168 0.59251785 0.60008788
 0.57869571

/Users/muhammadwaseem/Documents/GitHub/other-projects/Nature-inspired-Algorithms/model/PSO.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float32)


Global best: 0.2547530233860016
Personal best of all 20 particles:
[0.26636487 0.32332218 0.26777771 0.27683723 0.28294653 0.27387381
 0.28608075 0.2725763  0.25475302 0.27211091 0.31045923 0.28093535
 0.27501628 0.26240885 0.27122372 0.2809521  0.29994428 0.27195135
 0.27987185 0.26611879]
------------------------------------------------
Iteration: 11
Global best: 0.2547530233860016
Personal best of all 20 particles:
[0.26164594 0.32332218 0.26777771 0.26695997 0.26348591 0.26382086
 0.26453584 0.26280338 0.25475302 0.27211091 0.31045923 0.26544398
 0.26457748 0.25873318 0.2621184  0.26687402 0.29994428 0.26391476
 0.26656142 0.26150987]
------------------------------------------------
Iteration: 12
Global best: 0.24631528556346893
Personal best of all 20 particles:
[0.25471354 0.27442127 0.2629385  0.25369689 0.24631529 0.25449914
 0.25107628 0.25466478 0.25446638 0.26370138 0.26981923 0.25086963
 0.25445533 0.25502121 0.25467908 0.25909144 0.27071136 0.25446033
 0.25322294 0.2547036

Global best: 0.20758990943431854
Personal best of all 20 particles:
[0.2122058  0.22890872 0.21364719 0.21411763 0.21236265 0.21295376
 0.2118355  0.20758991 0.21223757 0.21051873 0.21209309 0.21247363
 0.2101063  0.21012291 0.21256016 0.21030553 0.21251093 0.21213694
 0.21139878 0.21204111]
------------------------------------------------
Iteration: 39
Global best: 0.2028421014547348
Personal best of all 20 particles:
[0.2122058  0.22890872 0.21364719 0.21411763 0.21236265 0.21295376
 0.2118355  0.2028421  0.21223757 0.21051873 0.21209309 0.21247363
 0.2101063  0.21012291 0.21256016 0.21030553 0.21251093 0.21213694
 0.21139878 0.21204111]
------------------------------------------------
Iteration: 40
Global best: 0.2008461207151413
Personal best of all 20 particles:
[0.2122058  0.22890872 0.21364719 0.21096894 0.21236265 0.21295376
 0.2118355  0.20084612 0.21223757 0.21051873 0.21209309 0.21247363
 0.20654728 0.20661849 0.21256016 0.20648903 0.21251093 0.21213694
 0.21139878 0.2120411

Global best: 0.16317620873451233
Personal best of all 20 particles:
[0.16821371 0.18971278 0.16502    0.16806459 0.18390228 0.18507013
 0.16402154 0.16454855 0.17038256 0.18731144 0.17582294 0.1657763
 0.16317621 0.18604867 0.18551877 0.16388483 0.16510279 0.18491161
 0.1634749  0.18487163]
------------------------------------------------
Iteration: 65
Global best: 0.16317620873451233
Personal best of all 20 particles:
[0.16821371 0.18971278 0.16502    0.16806459 0.18390228 0.18507013
 0.16402154 0.16454855 0.17020337 0.18731144 0.17582294 0.16556862
 0.16317621 0.18604867 0.18551877 0.16326748 0.16482361 0.18491161
 0.1634749  0.18487163]
------------------------------------------------
Iteration: 66
Global best: 0.16295282542705536
Personal best of all 20 particles:
[0.16821371 0.18971278 0.16502    0.16806459 0.18390228 0.18507013
 0.16402154 0.16454855 0.17020337 0.18731144 0.17582294 0.16556862
 0.16317621 0.18604867 0.18551877 0.16295283 0.16482361 0.18491161
 0.1634749  0.184871

Global best: 0.16171646118164062
Personal best of all 20 particles:
[0.16821371 0.18971278 0.16326252 0.16806459 0.18390228 0.18507013
 0.16258331 0.16268808 0.16579507 0.18731144 0.17582294 0.16382776
 0.16310777 0.18604867 0.18551877 0.16171646 0.16328427 0.18491161
 0.16204827 0.18487163]
------------------------------------------------
Iteration: 94
Global best: 0.16171646118164062
Personal best of all 20 particles:
[0.16821371 0.18971278 0.16326252 0.16806459 0.18390228 0.18507013
 0.16258331 0.16268808 0.16579507 0.18731144 0.17582294 0.16382776
 0.16310777 0.18604867 0.18551877 0.16171646 0.16328427 0.18491161
 0.16204827 0.18487163]
------------------------------------------------
Iteration: 95
Global best: 0.16171646118164062
Personal best of all 20 particles:
[0.16821371 0.18971278 0.16326252 0.16806459 0.18390228 0.18507013
 0.16258331 0.16268808 0.16579507 0.18731144 0.17582294 0.16382776
 0.16310777 0.18604867 0.18551877 0.16171646 0.16328427 0.18491161
 0.16204827 0.18487

## Evaluatio on test data (Ant Colony Optimization)

In [39]:
weights = atbw
weights = map_vertex_to_weight(graph, weights)
model = assign_weights(weights, model)

In [40]:
atbc # All time best cost

0.2959287464618683

In [41]:
y_test.shape

(1250, 1)

In [42]:
pred = (model.predict(X_test) >= 0.5).astype('int')[0] 
np.bincount(pred)

array([1162,   88])

In [43]:
target = y_test.reshape(-1)
np.bincount(target)

array([1130,  120])

In [44]:
(pred == target).mean()

0.9056

## Evaluation on test data (Particle Swarm Optimization)

In [20]:
## Best weights
index = np.argsort(final_loss)[0]
weights = final_particle[index, :]
model = assign_weights(weights, model)

In [21]:
final_loss[index]

0.16171646118164062

In [22]:
y_test.shape

(1250, 1)

In [23]:
pred = (model.predict(X_test) >= 0.5).astype('int')[0] 
np.bincount(pred)

array([1181,   69])

In [24]:
target = y_test.reshape(-1)
np.bincount(target)

array([1130,  120])

In [25]:
1169/1250

0.9352

## Evaluation on test data (Genetic Algorithm)

In [25]:
## Best weights
index = np.argsort(final_loss)[0]
weights = pop[index, :]
model = assign_weights(weights, model)

In [26]:
final_loss[index]

0.16221376

In [27]:
y_test.shape

(1250, 1)

In [28]:
pred = (model.predict(X_test) >= 0.5).astype('int')[0] 
np.bincount(pred)

array([1165,   85])

In [29]:
target = y_test.reshape(-1)
np.bincount(target)

array([1130,  120])

In [30]:
(pred == target).mean()

0.9384